In [1]:
import findspark
findspark.init(spark_home="/home/prabhakar/mybin/spark-3.0.2-bin-hadoop2.7-hive1.2")


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("handling complex data types").getOrCreate()


configs = spark.sparkContext.getConf().getAll()
for key, value in configs:
    print(f"{key}: {value}")

# Get the Spark UI URL
spark_ui_url = spark.sparkContext.uiWebUrl

print(f"Spark Job URL: {spark_ui_url}")

25/09/06 08:40:55 WARN Utils: Your hostname, prabhu resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/06 08:40:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/09/06 08:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/06 08:40:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


spark.driver.port: 34619
spark.app.id: local-1757148057859
spark.rdd.compress: True
spark.serializer.objectStreamReset: 100
spark.master: local[*]
spark.submit.pyFiles: 
spark.executor.id: driver
spark.submit.deployMode: client
spark.driver.host: 10.255.255.254
spark.app.name: handling complex data types
spark.ui.showConsoleProgress: true
spark.app.startTime: 1757148056486
Spark Job URL: http://10.255.255.254:4041


### **Benefits of Checkpointing**

Checkpointing provides several benefits, including:

**Improved Performance:** By truncating lineage chains, checkpointing reduces the overhead associated with recomputing lost partitions and improves the overall performance of your application
2
.

**Enhanced Fault Tolerance:** Checkpointing increases fault tolerance by persisting intermediate results to a reliable distributed file system, enabling faster recovery in case of node failures or data loss
2
.

**Simplified Debugging:** With shorter lineage chains, it becomes easier to track down and debug issues in your data processing pipeline
2

### **Cons of Checkpointing**
DAG lineage after the checkpoint doesnt have the lineage information before the checkpoint.
It's only information is the the checkpointted information

In [4]:
df = spark.read.format("jdbc").option("driver", "com.mysql.cj.jdbc.Driver").option("user", "sparkuser").option("password", "prabharkar").option("url", "jdbc:mysql://172.22.192.1:3306/batch11").option("dbtable", "cards").load()
df.show()

+-----+-----+
|empid| name|
+-----+-----+
|    1| john|
|    2|steve|
|    3|  bob|
+-----+-----+



In [ ]:
from pyspark.sql.functions import col, upper
df1=df.withColumn("caps_holdername", upper(col("holdername")))
df2=df1.drop(col("holdername")).withColumnRenamed("caps_holdername", "holdername")
df2.select("cid", "holdername", "card_num", "cvv", "pin").write.format("jdbc").option("driver", "com.mysql.cj.jdbc.Driver").option("user", "sparkuser").option("password", "prabharkar").option("url", "jdbc:mysql://172.22.192.1:3306/batch11").option("dbtable", "cards").save()

### Using FIle

### Mock data

In [21]:
data=[
    (1, "john"),
    (2, "steve"),
    (3, "bob")
]

fsdf=spark.createDataFrame(data, ["empid", "name"])
fsdf.show()
fsdf.coalesce(1).write.mode("overwrite").option("header", "true").csv("/home/prabhakar/data/emp_data")

+-----+-----+
|empid| name|
+-----+-----+
|    1| john|
|    2|steve|
|    3|  bob|
+-----+-----+



In [27]:


fdf=spark.read.option("header", "true").csv("/home/prabhakar/data/emp_data/emp.csv")
fdf.show()

+-----+-----+
|empid| name|
+-----+-----+
|    1| john|
|    2|steve|
|    3|  bob|
+-----+-----+



In [28]:
from pyspark.sql.functions import col, upper
fdf1 = fdf.withColumn("captial_name", upper(col("name")))
fdf2 = fdf1.drop(col("name")).withColumnRenamed("captial_name", "name")

spark.sparkContext.setCheckpointDir("/home/prabhakar/data/checkpoint_data")

fdf3=fdf2

fdf1.show()
fdf2.show()

+-----+-----+------------+
|empid| name|captial_name|
+-----+-----+------------+
|    1| john|        JOHN|
|    2|steve|       STEVE|
|    3|  bob|         BOB|
+-----+-----+------------+

+-----+-----+
|empid| name|
+-----+-----+
|    1| JOHN|
|    2|STEVE|
|    3|  BOB|
+-----+-----+



In [29]:
fdf2.write.mode("overwrite").option("header", "true").csv("/home/prabhakar/data/emp_data")

25/09/06 10:39:58 ERROR Utils: Aborting task
java.io.FileNotFoundException: File file:/home/prabhakar/data/emp_data/emp.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at or

Py4JJavaError: An error occurred while calling o357.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.s